# Globe

see: Globe.js file in Cesium source code.

# CesiumTerrainProvider

`::requestTileGeometry(x, y, level, request)` this func. first fetch raw terrain data(float32Array of its type) from server, then create `HeightmapTerrainData` instance the moment promise fulfilled. Pseudo code looks like:

```js
function requestTileGeometry(...){
    const promise = new Promise((...)=>{
        // load terrain data from server.
    });
    return promise.then(function (buffer) {
        return createHeightmapTerrainData(provider, buffer, level, x, y);
    });
}
```

I think we may use `CustomHeightmapTerrainProvider` to provider a better control under terrain data. `ArcGISTiledElevationTerrainProvider` is also a good choose.

# GlobeSurfaceTileProvider / GlobeSurfaceTile

Class `GlobeSurfaceTileProvider` produces `GlobeSurfaceTile` instance, this instance then call `CesiumTerrainProvider::requestTileGeometry(...)` to try to get terrain data (instance of `HeightmapTerrainData`), and mount to a variable called `terrainData`.

At some point, e.g. need to be rendered on screen, `GlobeSurfaceTile` then call `::transform()` to gen a `TerrainMesh` instance from previously mounted `terrainData`. And as u figured out that this instance then also mounted onto `GlobeSurfaceTile` with a name called `mesh`.

finally, `GlobeSurfaceTile` has at least two member variables, which are:
* `terrainData` of type `HeightmapTerrainData`;
* `mesh` of type `TerrainMesh`;

Pseudo code looks like:

```js
let _tile = new GlobeSurfaceTile(...);
_provider.requestTileGeometry(...).then(data =>{
    _tile.terrainData = data; /// instance of HeightmapTerrainData;
});

/// if need further process.
_tile.transform();

function transform(){
    // ...
    _tile.terrainData.createMesh().then(mesh => {
        _tile.mesh = mesh;
    });
}
```

# QuadtreePrimitive

this class possesses the root tile and a reference to `GlobeSurfaceTileProvider`, at the correct time, provider gen `GlobeSurfaceTile` and mount to tile.data.

**NOTICE**

Do not be confused by `tile`, there are at least 2 tiles, one is `GlobeSufraceTile`, and another one is `QuadtreeTile`. `QuadtreePrimitive` possesses the root tile which is `QuadtreeTile` not the one that instantiated from `GlobeSurfaceTile`.

```js
/// ::render(frameState) function.
if (passes.render) {
    tileProvider.beginUpdate(frameState);

    const tilesToRender = selectTilesForRendering(this, frameState);
    for (let i = 0, len = tilesToRender.length; i < len; ++i) {
        const tile = tilesToRender[i];
        tileProvider.showTileThisFrame(tile, frameState);
    }

    tileProvider.endUpdate(frameState);
}
// ...
```

# ImagerLayer

This's a class that handles a set of same origin imageries. Different imagery provider offers different piece of imagy. Firstly, this class get raw imagery data from Internet through `ImageryProvider`, and then parse into a instance of class `Imagery`.